In [2]:
import redis

In [3]:
db = redis.Redis(host='localhost', port=6379, db=11)

In [4]:
keys = db.keys()

In [5]:
len(keys)

187385

In [6]:
import json

In [7]:
keys[0]

b'tenggok'

In [14]:
db['agung']

b'{"definisi": ["[a.gung]  |  \\u0627\\u0762\\u0648\\u06a0Definisi : 1. (paling) besar, terutama: balai ~; layar ~; tiang ~; 2. yg tertinggi: pemerintah ~; 3. teristimewa, terkemuka, terutama: Allahyarham Tun Abdul Razak seorang pemimpin yg ~; Allahyarham P. Ramlee seniman ~ Malaysia; tamu ~; 4. yg terbuka kpd dan utk dihadiri oleh semua ahli (bkn mesyuarat, persidangan, dsb): persidangan ~ UMNO; mesyuarat ~ Kesatuan Sekerja DBP; mengagungkan menjadikan (menganggap dsb) agung, membesarkan, memuliakan, me\\u00adninggikan: al-Quran ~ ilmu pengetahuan dan memuliakan ahli ilmu; mengagung-agungkan selalu (sering kali) mengagungkan, sangat mengagungkan; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; pengagungan perbuatan dsb mengagungkan: antara yg menghalang perkembangan pesat bahasa Melayu ialah kekurangan bahan rujuk\\

In [17]:
d = json.loads(db[keys[0]])

In [32]:
from tqdm import tqdm

selected = {}

for k in tqdm(keys):
    d = json.loads(db[k])
    if not len(d['definisi']):
        continue
        
    selected[k.decode()] = d

100%|████████████████████████████████████████████████████████████████████████████████| 187385/187385 [00:05<00:00, 31504.46it/s]


In [31]:
len(json.loads(db['agung'])['definisi'])

3

In [25]:
len(selected)

31589

In [41]:
import re
from unidecode import unidecode

alphabet = 'qwertyuiopasdfghjklzxcvbnm'
consonants = 'bcdfghjklmnpqrstvwxyz'
vowels = 'aeiou'
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

In [42]:
import enchant
d = enchant.Dict('en_US')

In [71]:
results = set()

for data in tqdm(selected.values()):
    
    texts = []
    for definisi in data['definisi']:
        texts.extend(definisi.split('Definisi : ')[1].split())

    if data['tesaurus']:
        texts.extend(data['tesaurus'])
    
    for k in range(len(texts)):
        
        t = texts[k]
        t = unidecode(t)
        t = t.replace('--', '-')
        for p in PUNCTUATION:
            t = t.replace(p, '')

        if all([c in consonants for c in t]):
            continue

        if len(t) <= 3:
            continue

        if len(re.findall(r'\d+', t)):
            continue

        if t[0] in '-':
            continue

        if t in ['yg', 'utk']:
            continue
            
        if t[-1] in '-':
            continue
        
        if 'dll' in t:
            continue
            
        t = t.lower()
        
        if d.check(t):
            continue
            
        if t in results:
            continue
            
        results.add(t)

100%|███████████████████████████████████████████████████████████████████████████████████| 31589/31589 [00:08<00:00, 3776.55it/s]


In [72]:
len(results)

49640

In [74]:
with open('dbp-v3.json', 'w') as fopen:
    json.dump(selected, fopen)

In [75]:
with open('malay-words-v3.json', 'w') as fopen:
    json.dump(list(results), fopen)